In [1]:
import keras_tuner as kt
from tensorflow import keras

In [4]:
fashion_mnist = keras.datasets.fashion_mnist
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from 'c:\\users\\ameya gupta\\miniconda3\\lib\\site-packages\\tensorflow\\keras\\datasets\\fashion_mnist\\__init__.py'>

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

# Main model

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

# Finding best parameters

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 05m 33s]
val_accuracy: 0.9151666760444641

Best val_accuracy So Far: 0.9151666760444641
Total elapsed time: 01h 00m 19s
INFO:tensorflow:Oracle triggered exit


In [16]:
model = tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3469424   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,489,882
Trainable params: 3,489,882
Non-trainable params: 0
_________________________________________________________________


# Using best model

In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 96s 56ms/step - loss: 0.1414 - accuracy: 0.9464 - val_loss: 0.2594 - val_accuracy: 0.9137
Epoch 5/10
1688/1688 [==============================] - 97s 57ms/step - loss: 0.1058 - accuracy: 0.9599 - val_loss: 0.2849 - val_accuracy: 0.9150
Epoch 6/10
1688/1688 [==============================] - 96s 57ms/step - loss: 0.0750 - accuracy: 0.9721 - val_loss: 0.3427 - val_accuracy: 0.9102
Epoch 7/10
1688/1688 [==============================] - 109s 64ms/step - loss: 0.0558 - accuracy: 0.9798 - val_loss: 0.3575 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 132s 78ms/step - loss: 0.0419 - accuracy: 0.9846 - val_loss: 0.3838 - val_accuracy: 0.9090
Epoch 9/10
1688/1688 [==============================] - 133s 79ms/step - loss: 0.0347 - accuracy: 0.9876 - val_loss: 0.4264 - val_accuracy: 0.9150
Epoch 10/10
1688/1688 [==============================] - 133s 79ms/step - loss: 0.0275 - accuracy: 0.9903 - val_loss: 0.4

In [21]:
# accuray is fairly good